# Análisis del Mercado Laboral Español (EPA)

Proyecto de análisis exploratorio de datos (EDA) sobre el mercado laboral español
utilizando datos abiertos del Instituto Nacional de Estadística (INE) — Encuesta de Población Activa (EPA).

## Objetivo

Entender la estructura del mercado laboral español por provincia, sexo y sector económico,
su evolución trimestral, y las desigualdades territoriales y de género en empleo y desempleo.

## Preguntas

- **Q1:** ¿Cuáles son las provincias con mayor y menor tasa de paro? ¿Ha cambiado el ranking?
- **Q2:** ¿Existe brecha de género en las tasas de actividad, empleo y paro? ¿Varía por provincia?
- **Q3:** ¿Cómo se distribuye el empleo por sector económico (agricultura, industria, construcción, servicios) y cómo varía geográficamente?
- **Q4:** ¿Cómo ha evolucionado el empleo total a lo largo del periodo?
- **Q5:** ¿Existe estacionalidad en el empleo/paro (trimestral)? ¿Afecta más a unas provincias que a otras?
- **Q6:** ¿Cómo varía la tasa de paro según el grupo de edad? ¿Qué diferencias hay por sexo?
- **Q7:** ¿Cómo ha evolucionado el paro juvenil frente al total?
- **Q8:** ¿Existe brecha de desempleo entre trabajadores españoles y extranjeros? ¿Varía según la edad?

In [ ]:
import json
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

sns.set_theme(context='notebook', style='whitegrid')
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 60)

# Add project root to sys.path (notebook runs from notebooks/)
sys.path.insert(0, str(Path.cwd().parent))

# Import centralized paths from src/config.py
from src.config import ROOT, DATA_RAW, DATA_PROCESSED, CHARTS_DIR, RAW_PATH, OUT_PATH

CHARTS = CHARTS_DIR  # alias used throughout this notebook
CHARTS.mkdir(parents=True, exist_ok=True)

# Clean previous charts before generating new ones
for f in CHARTS.glob('*.png'):
    f.unlink()
print('Limpiados graficos anteriores.')

csv_path = RAW_PATH
print(f'Project root: {ROOT}')
print(f'CSV path: {csv_path}')
print(f'Exists: {csv_path.exists()}')

## 1) QC Rapido

Detectar problemas sin perder 30 minutos: `head()`, `shape`, `info()`, nulos, duplicados.

In [ ]:
df = pd.read_csv(csv_path)

display(df.head(10))
print(f'\nShape: {df.shape}')
print(f'\nInfo:')
display(df.info())

na_rate = (df.isna().mean().sort_values(ascending=False) * 100).round(2)
dup_count = df.duplicated().sum()

print(f'\nMissing %:')
display(na_rate)
print(f'\nDuplicated rows: {dup_count}')

In [ ]:
# Cardinality & rare categories check
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f'Categorical columns: {cat_cols}\n')

for col in cat_cols:
    nunique = df[col].nunique(dropna=True)
    print(f'--- {col} --- (n_unique={nunique})')
    display(df[col].value_counts(dropna=False).head(10))
    print()

### Problemas detectados

| Problema | Detalle |
|---|---|
| Nombres de columna sucios | Espacios, mayusculas mezcladas (`' Valor'`, `'Fecha '`) |
| Valor como texto | Mezcla de numeros con punto y coma decimal (`42315.3` vs `203,2`) |
| Fechas en 5 formatos | `2024-06-30`, `30/06/2025`, `2025/03/31`, `Dec 31, 2024`, `1727733600000` (ms) |
| Serie_nombre inconsistente | Mismo dato en mayusculas/minusculas, espacios extra |
| ~3% de nulls en Valor | 1125 valores faltantes |
| 20 filas duplicadas | Filas repetidas exactas |
| Serie_nombre empaquetado | Todas las dimensiones (provincia, sexo, actividad) en un solo string |

## 2) Limpieza

In [ ]:
df_clean = df.copy()

# 2.1) Fix column names: strip, lowercase, underscores
df_clean.columns = [c.strip().lower().replace(' ', '_') for c in df_clean.columns]
print('Columns after fix:', df_clean.columns.tolist())

In [ ]:
# 2.2) Fix 'valor': replace commas with dots, convert to numeric
df_clean['valor'] = (
    df_clean['valor']
    .astype('string')
    .str.replace(',', '.', regex=False)
)
df_clean['valor'] = pd.to_numeric(df_clean['valor'], errors='coerce')
print(f'valor dtype: {df_clean["valor"].dtype}')
print(f'valor nulls: {df_clean["valor"].isna().sum()}')

In [ ]:
# 2.3) Fix 'serie_nombre': strip whitespace, normalize to title case
df_clean['serie_nombre'] = (
    df_clean['serie_nombre']
    .astype('string')
    .str.strip()
)

# Normalize: all to the canonical form (first letter caps, rest lower)
# We need to be careful with proper nouns and acronyms
# Strategy: lowercase everything, then fix known patterns
df_clean['serie_nombre_lower'] = df_clean['serie_nombre'].str.lower()

# Check how many unique after lowercasing
print(f'Unique serie_nombre (raw):   {df_clean["serie_nombre"].nunique()}')
print(f'Unique serie_nombre (lower): {df_clean["serie_nombre_lower"].nunique()}')

In [ ]:
# 2.4) Parse serie_nombre into structured columns
# Patterns detected (all lowered for matching):
#   Table 65345 national: "total nacional. {sexo}. 16 y más años. {actividad}. personas."
#   Table 65345 province: "{sexo}. {provincia}. 16 y más años. {actividad}. personas."
#   Table 65349: "tasa de {tipo}. {provincia|ambos sexos}. {sexo|total nacional}. total."
#   Table 65354: "{provincia|total nacional}. ocupados. ambos sexos. {sector}. personas."

# Build canonical province name mapping — prefer properly-cased (non-UPPER) names
provincia_canonical = {}
for nombre in df_clean[df_clean['tabla'] == 65345]['serie_nombre'].unique():
    parts = [p.strip() for p in nombre.split('. ') if p.strip()]
    if parts[0].lower() != 'total nacional' and len(parts) >= 4:
        prov = parts[1]
        key = prov.lower()
        # Only overwrite if we don't have a canonical form yet, or the current one is ALL CAPS
        if key not in provincia_canonical or provincia_canonical[key] == provincia_canonical[key].upper():
            provincia_canonical[key] = prov

# Manual fixes for provinces that may only appear uppercased
provincia_fixes = {
    'albacete': 'Albacete', 'alicante/alacant': 'Alicante/Alacant',
    'araba/álava': 'Araba/Álava', 'asturias': 'Asturias', 'badajoz': 'Badajoz',
    'barcelona': 'Barcelona', 'burgos': 'Burgos', 'cantabria': 'Cantabria',
    'castellón/castelló': 'Castellón/Castelló', 'ceuta': 'Ceuta',
    'ciudad real': 'Ciudad Real', 'coruña, a': 'Coruña, A', 'cuenca': 'Cuenca',
    'gipuzkoa': 'Gipuzkoa', 'girona': 'Girona', 'guadalajara': 'Guadalajara',
    'lugo': 'Lugo', 'melilla': 'Melilla', 'murcia': 'Murcia',
    'málaga': 'Málaga', 'navarra': 'Navarra', 'palmas, las': 'Palmas, Las',
    'rioja, la': 'Rioja, La', 'salamanca': 'Salamanca', 'segovia': 'Segovia',
    'teruel': 'Teruel', 'valencia/valència': 'Valencia/València',
    'valladolid': 'Valladolid', 'zamora': 'Zamora', 'zaragoza': 'Zaragoza',
}
for k, v in provincia_fixes.items():
    if k not in provincia_canonical or provincia_canonical[k] == provincia_canonical[k].upper():
        provincia_canonical[k] = v

provincia_canonical['total nacional'] = 'Total Nacional'

# Canonical sexo mapping
sexo_canonical = {'ambos sexos': 'Ambos sexos', 'hombres': 'Hombres', 'mujeres': 'Mujeres'}

def canon_prov(s):
    return provincia_canonical.get(s.lower().strip(), s.strip().title())

def canon_sexo(s):
    return sexo_canonical.get(s.lower().strip(), s.strip().title())

def parse_serie_65345(nombre_lower):
    parts = [p.strip() for p in nombre_lower.split('. ') if p.strip()]
    if parts[0] == 'total nacional':
        return {'provincia': 'Total Nacional', 'sexo': canon_sexo(parts[1]), 'actividad': parts[3].title()}
    else:
        return {'provincia': canon_prov(parts[1]), 'sexo': canon_sexo(parts[0]), 'actividad': parts[3].title()}

def parse_serie_65349(nombre_lower):
    parts = [p.strip() for p in nombre_lower.split('. ') if p.strip()]
    tasa = next((p for p in parts if 'tasa' in p), 'desconocida')
    tasa_display = tasa.replace('tasa de ', 'Tasa de ').replace('la población', 'la poblacion')
    remaining = [p for p in parts if p != tasa and p not in ('total', 'total.', 'personas')]
    provincia = 'Total Nacional'
    sexo = 'Ambos sexos'
    for r in remaining:
        r_clean = r.rstrip('.')
        if r_clean in ('ambos sexos', 'hombres', 'mujeres'):
            sexo = canon_sexo(r_clean)
        elif r_clean == 'total nacional':
            provincia = 'Total Nacional'
        elif r_clean not in ('total',):
            provincia = canon_prov(r_clean)
    return {'provincia': provincia, 'sexo': sexo, 'actividad': tasa_display}

def parse_serie_65354(nombre_lower):
    parts = [p.strip() for p in nombre_lower.split('. ') if p.strip()]
    provincia = canon_prov(parts[0]) if parts[0] != 'ocupados' else 'Total Nacional'
    sector = 'Total'
    for p in parts:
        if p in ('agricultura', 'industria', 'construcción', 'servicios', 'total cnae'):
            sector = p.title()
            break
    return {'provincia': provincia, 'sexo': 'Ambos sexos', 'actividad': f'Ocupados - {sector}'}

# Apply parsing using lowered names
records = []
for _, row in df_clean.iterrows():
    nombre_lower = row['serie_nombre_lower']
    tabla = row['tabla']
    try:
        if tabla == 65345:
            parsed = parse_serie_65345(nombre_lower)
        elif tabla == 65349:
            parsed = parse_serie_65349(nombre_lower)
        elif tabla == 65354:
            parsed = parse_serie_65354(nombre_lower)
        else:
            parsed = {'provincia': 'Desconocida', 'sexo': 'Desconocido', 'actividad': 'Desconocida'}
    except Exception:
        parsed = {'provincia': 'Error', 'sexo': 'Error', 'actividad': 'Error'}
    records.append(parsed)

df_parsed = pd.DataFrame(records)
df_clean = pd.concat([df_clean, df_parsed], axis=1)

print(f'New columns: {df_parsed.columns.tolist()}')
print(f'\nProvincia unique: {df_clean["provincia"].nunique()}')
print(f'Sexo unique: {df_clean["sexo"].nunique()}')
print(f'Actividad unique: {df_clean["actividad"].nunique()}')
print()
print('Provincias sample:')
display(df_clean['provincia'].value_counts().head(15))
print('\nActividad values:')
display(df_clean['actividad'].value_counts())

In [ ]:
# 2.5) Parse dates robustly
from datetime import datetime

def parse_fecha(val):
    """Parse date from mixed formats: ISO, dd/mm/yyyy, yyyy/mm/dd, textual, ms timestamp."""
    if pd.isna(val) or val == '<NA>':
        return pd.NaT
    val = str(val).strip()
    # Try ms timestamp first (pure digits, very long)
    if val.isdigit() and len(val) > 10:
        try:
            return pd.Timestamp(int(val), unit='ms')
        except Exception:
            pass
    # Try standard formats
    for fmt in ['%Y-%m-%d', '%d/%m/%Y', '%Y/%m/%d', '%b %d, %Y']:
        try:
            return pd.Timestamp(datetime.strptime(val, fmt))
        except (ValueError, AttributeError):
            continue
    # Fallback
    try:
        return pd.to_datetime(val, dayfirst=True)
    except Exception:
        return pd.NaT

# Build a Series from parsed values and convert to datetime (satisfies type-checkers)
parsed = [parse_fecha(x) for x in df_clean['fecha']]
df_clean['fecha'] = pd.to_datetime(pd.Series(parsed))
print(f'fecha dtype: {df_clean["fecha"].dtype}')
print(f'fecha nulls: {df_clean["fecha"].isna().sum()}')
print(f'fecha range: {df_clean["fecha"].min()} to {df_clean["fecha"].max()}')

In [ ]:
# 2.6) Normalize categorical values

# Normalize sexo
sexo_map = {
    'ambos sexos': 'Ambos sexos',
    'hombres': 'Hombres',
    'mujeres': 'Mujeres',
    'AMBOS SEXOS': 'Ambos sexos',
    'HOMBRES': 'Hombres',
    'MUJERES': 'Mujeres',
}
df_clean['sexo'] = df_clean['sexo'].str.strip().replace(sexo_map)

# Normalize provincia
df_clean['provincia'] = df_clean['provincia'].str.strip()

# Normalize actividad
df_clean['actividad'] = df_clean['actividad'].str.strip()

print('Sexo values:')
display(df_clean['sexo'].value_counts())
print('\nProvincia (top 10):')
display(df_clean['provincia'].value_counts().head(10))

In [ ]:
# 2.7) Drop duplicates
print(f'Before dedup: {df_clean.shape[0]}')
df_clean = df_clean.drop_duplicates(subset=['tabla', 'serie_cod', 'anyo', 'periodo_id'])
print(f'After dedup:  {df_clean.shape[0]}')

# 2.8) Drop helper columns
df_clean = df_clean.drop(columns=['serie_nombre_lower', 'secreto'], errors='ignore')

print(f'\nFinal clean shape: {df_clean.shape}')
print(f'Columns: {df_clean.columns.tolist()}')
display(df_clean.head())

In [ ]:
# Validation asserts
assert df_clean.duplicated(subset=['tabla', 'serie_cod', 'anyo', 'periodo_id']).sum() == 0, 'Duplicates remain!'
assert pd.api.types.is_numeric_dtype(df_clean['valor']), f'valor is not numeric! dtype={df_clean["valor"].dtype}'
assert pd.api.types.is_datetime64_any_dtype(df_clean['fecha']), 'fecha is not datetime!'
assert df_clean['sexo'].isin(['Ambos sexos', 'Hombres', 'Mujeres']).all(), f'sexo has unexpected values: {df_clean["sexo"].unique()}'
print('All validations passed.')

## 3) Feature Engineering

In [ ]:
df_feat = df_clean.copy()

# 3.1) Temporal features
df_feat['trimestre'] = df_feat['fecha'].dt.to_period('Q').astype('string')
df_feat['mes'] = df_feat['fecha'].dt.month
df_feat['year'] = df_feat['fecha'].dt.year

# 3.2) Map periodo_id to quarter label
periodo_map = {19: 'T4', 20: 'T1', 21: 'T2', 22: 'T3'}
df_feat['trimestre_label'] = df_feat['periodo_id'].map(periodo_map).fillna('Otro')

# 3.3) Classify table source
tabla_map = {
    65345: 'Poblacion',
    65349: 'Tasas',
    65354: 'Ocupados por sector',
}
df_feat['fuente'] = df_feat['tabla'].map(tabla_map)

# 3.4) Flag national vs provincial
df_feat['es_nacional'] = df_feat['provincia'].str.lower().str.contains('total nacional', na=False)

# 3.5) Map provinces to Autonomous Communities (CCAA)
ccaa_map = {
    'Almería': 'Andalucía', 'Cádiz': 'Andalucía', 'Córdoba': 'Andalucía',
    'Granada': 'Andalucía', 'Huelva': 'Andalucía', 'Jaén': 'Andalucía',
    'Málaga': 'Andalucía', 'Sevilla': 'Andalucía',
    'Huesca': 'Aragón', 'Teruel': 'Aragón', 'Zaragoza': 'Aragón',
    'Asturias': 'Asturias',
    'Balears, Illes': 'Illes Balears',
    'Palmas, Las': 'Canarias', 'Santa Cruz de Tenerife': 'Canarias',
    'Cantabria': 'Cantabria',
    'Ávila': 'Castilla y León', 'Burgos': 'Castilla y León', 'León': 'Castilla y León',
    'Palencia': 'Castilla y León', 'Salamanca': 'Castilla y León',
    'Segovia': 'Castilla y León', 'Soria': 'Castilla y León',
    'Valladolid': 'Castilla y León', 'Zamora': 'Castilla y León',
    'Albacete': 'Castilla-La Mancha', 'Ciudad Real': 'Castilla-La Mancha',
    'Cuenca': 'Castilla-La Mancha', 'Guadalajara': 'Castilla-La Mancha',
    'Toledo': 'Castilla-La Mancha',
    'Barcelona': 'Cataluña', 'Girona': 'Cataluña', 'Lleida': 'Cataluña',
    'Tarragona': 'Cataluña',
    'Alicante/Alacant': 'Comunitat Valenciana', 'Castellón/Castelló': 'Comunitat Valenciana',
    'Valencia/València': 'Comunitat Valenciana',
    'Badajoz': 'Extremadura', 'Cáceres': 'Extremadura',
    'Coruña, A': 'Galicia', 'Lugo': 'Galicia', 'Ourense': 'Galicia',
    'Pontevedra': 'Galicia',
    'Madrid': 'Comunidad de Madrid',
    'Murcia': 'Región de Murcia',
    'Navarra': 'Navarra',
    'Araba/Álava': 'País Vasco', 'Bizkaia': 'País Vasco', 'Gipuzkoa': 'País Vasco',
    'Rioja, La': 'La Rioja',
    'Ceuta': 'Ceuta', 'Melilla': 'Melilla',
    'Total Nacional': 'Total Nacional',
}
df_feat['ccaa'] = df_feat['provincia'].map(ccaa_map).fillna('Desconocida')

print(f'Shape with features: {df_feat.shape}')
print(f'New columns: trimestre, mes, year, trimestre_label, fuente, es_nacional, ccaa')
display(df_feat.head())
print(f'\nCCAA unique: {df_feat["ccaa"].nunique()}')
print(f'CCAA values:')
display(df_feat['ccaa'].value_counts())

In [ ]:
# Derive the period from the data itself — used in all chart titles
PERIOD_START = int(df_feat['anyo'].min())
PERIOD_END = int(df_feat['anyo'].max())
PERIOD_LABEL = f'{PERIOD_START}–{PERIOD_END}'
print(f'Data period: {PERIOD_LABEL}')

## 4) Guardar datos procesados

In [ ]:
out_path = OUT_PATH
out_path.parent.mkdir(parents=True, exist_ok=True)
df_feat.to_csv(out_path, index=False)
print(f'Saved: {out_path}')
print(f'Shape: {df_feat.shape}')

## 5) Visualizaciones

4-6 graficos con intencion: comparacion, distribucion, relacion, temporal.

In [ ]:
# =====================================================
# CHART 1: Tasa de paro por provincia (ultimo trimestre)
# =====================================================

# Filter: rates table, unemployment rate, both sexes, provincial (not national)
df_tasas = df_feat[
    (df_feat['tabla'] == 65349) &
    (df_feat['actividad'].str.contains('paro', case=False, na=False)) &
    (df_feat['sexo'] == 'Ambos sexos') &
    (~df_feat['es_nacional'])
].copy()

# Get the most recent quarter
ultimo_trimestre = df_tasas['trimestre'].dropna().max()
df_ultimo = df_tasas[df_tasas['trimestre'] == ultimo_trimestre].dropna(subset=['valor']).copy()

print(f'Ultimo trimestre: {ultimo_trimestre}')
print(f'Provincias: {df_ultimo["provincia"].nunique()}')

# Sort by unemployment rate
df_plot = df_ultimo.sort_values('valor', ascending=True)

fig, ax = plt.subplots(figsize=(10, 14))
median_val = df_plot['valor'].median()
colors = ['#e74c3c' if v > median_val else '#2ecc71' for v in df_plot['valor'].values]
ax.barh(df_plot['provincia'], df_plot['valor'], color=colors)
ax.set_xlabel('Tasa de paro (%)')
ax.set_title(f'Tasa de paro por provincia — {ultimo_trimestre} ({PERIOD_LABEL})\n(rojo = por encima de la mediana)')
ax.axvline(median_val, color='gray', linestyle='--', alpha=0.7, label='Mediana')
ax.legend()
plt.tight_layout()
fig.savefig(CHARTS / '01_tasa_paro_por_provincia.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\nTop 5 paro:\n{df_plot.tail(5)[["provincia", "valor"]].to_string(index=False)}')
print(f'\nBottom 5 paro:\n{df_plot.head(5)[["provincia", "valor"]].to_string(index=False)}')

In [ ]:
# =====================================================
# CHART 2: Brecha de genero en tasa de paro (H vs M)
# =====================================================

df_genero = df_feat[
    (df_feat['tabla'] == 65349) &
    (df_feat['actividad'].str.contains('paro', case=False, na=False)) &
    (df_feat['sexo'].isin(['Hombres', 'Mujeres'])) &
    (df_feat['es_nacional'])
].copy()

df_genero = df_genero.sort_values('fecha')

fig, ax = plt.subplots(figsize=(12, 5))
for sexo, color in [('Hombres', '#3498db'), ('Mujeres', '#e74c3c')]:
    mask = df_genero['sexo'] == sexo
    ax.plot(df_genero.loc[mask, 'fecha'], df_genero.loc[mask, 'valor'],
            marker='o', label=sexo, color=color, linewidth=2)

ax.set_xlabel('Fecha')
ax.set_ylabel('Tasa de paro (%)')
ax.set_title(f'Evolucion de la tasa de paro por sexo — Total Nacional ({PERIOD_LABEL})')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
fig.savefig(CHARTS / '02_brecha_genero_paro.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =====================================================
# CHART 3: Empleo por sector economico (evolucion)
# =====================================================

df_sector = df_feat[
    (df_feat['tabla'] == 65354) &
    (df_feat['es_nacional']) &
    (~df_feat['actividad'].str.contains('Total', case=False, na=False))
].copy()

# Extract sector name from actividad
df_sector['sector'] = df_sector['actividad'].str.replace('Ocupados - ', '', regex=False)
df_sector = df_sector.sort_values('fecha')

fig, ax = plt.subplots(figsize=(12, 5))
sector_colors = {'Agricultura': '#27ae60', 'Industria': '#2980b9',
                 'Construcción': '#f39c12', 'Servicios': '#8e44ad'}
for sector in df_sector['sector'].unique():
    mask = df_sector['sector'] == sector
    color = sector_colors.get(sector, 'gray')
    ax.plot(df_sector.loc[mask, 'fecha'], df_sector.loc[mask, 'valor'], # type: ignore
            marker='o', label=sector, color=color, linewidth=2)

ax.set_xlabel('Fecha')
ax.set_ylabel('Ocupados (miles de personas)')
ax.set_title(f'Evolucion del empleo por sector economico — Total Nacional ({PERIOD_LABEL})')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
fig.savefig(CHARTS / '03_empleo_por_sector.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =====================================================
# CHART 4: Distribucion media de ocupados por provincia
# =====================================================

df_ocup = df_feat[
    (df_feat['tabla'] == 65345) &
    (df_feat['actividad'].str.lower() == 'ocupados') &
    (df_feat['sexo'] == 'Ambos sexos') &
    (~df_feat['es_nacional'])
].dropna(subset=['valor']).copy()

avg = df_ocup.groupby('provincia')['valor'].mean().sort_values(ascending=True)

fig, ax = plt.subplots(figsize=(10, 14))
median_val = avg.median()
colors = ['#2980b9' if v < median_val else '#e67e22' for v in avg.values]
ax.barh(avg.index, avg.values, color=colors)
ax.set_xlabel('Ocupados (miles, media del periodo)')
ax.set_title(f'Distribucion media de ocupados por provincia ({PERIOD_LABEL})')
plt.tight_layout()
fig.savefig(CHARTS / '04_distribucion_ocupados.png', dpi=150, bbox_inches='tight')
plt.show()

print('Altamente sesgada: la mayoria de provincias tienen poca poblacion,'
      ' pero Madrid y Barcelona concentran el empleo.')

In [ ]:
# =====================================================
# CHART 5: Evolucion del empleo total (ocupados total nacional)
# =====================================================

df_empleo = df_feat[
    (df_feat['tabla'] == 65345) &
    (df_feat['actividad'].str.lower() == 'ocupados') &
    (df_feat['sexo'] == 'Ambos sexos') &
    (df_feat['es_nacional'])
].sort_values('fecha').copy()

vmin, vmax = df_empleo['valor'].min(), df_empleo['valor'].max()
buffer = (vmax - vmin) * 0.10
y_bottom = vmin - buffer

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(df_empleo['fecha'], df_empleo['valor'], marker='o', color='#2c3e50', linewidth=2)
ax.fill_between(df_empleo['fecha'], df_empleo['valor'], y_bottom, alpha=0.15, color='#2c3e50')
ax.set_ylim(bottom=y_bottom)

ax.set_xlabel('Fecha')
ax.set_ylabel('Ocupados (miles de personas)')
ax.set_title(f'Evolucion del empleo total — Total Nacional ({PERIOD_LABEL})')
ax.grid(True, alpha=0.3)
plt.tight_layout()
fig.savefig(CHARTS / '05_evolucion_empleo_total.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =====================================================
# CHART 6: Heatmap — tasa de paro por CCAA y trimestre
# =====================================================

# Aggregate: mean unemployment rate by CCAA and trimestre
df_heat = df_feat[
    (df_feat['tabla'] == 65349) &
    (df_feat['actividad'].str.contains('paro', case=False, na=False)) &
    (df_feat['sexo'] == 'Ambos sexos') &
    (~df_feat['es_nacional']) &
    (df_feat['ccaa'] != 'Desconocida')
].dropna(subset=['valor']).copy()

# Ensure valor is plain float (not nullable Float64)
df_heat['valor'] = df_heat['valor'].astype(float)

pivot = df_heat.pivot_table(values='valor', index='ccaa', columns='trimestre',
                            aggfunc='mean')
# Sort by mean across all quarters
pivot = pivot.loc[pivot.mean(axis=1).sort_values(ascending=False).index]

fig, ax = plt.subplots(figsize=(14, 10))
sns.heatmap(pivot, annot=False, cmap='RdYlGn_r', ax=ax,
            linewidths=0.5, cbar_kws={'label': 'Tasa de paro (%)'})
ax.set_title(f'Tasa de paro media por CCAA y trimestre ({PERIOD_LABEL})')
ax.set_xlabel('Trimestre')
ax.set_ylabel('Comunidad Autonoma')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
fig.savefig(CHARTS / '06_heatmap_paro_ccaa.png', dpi=150, bbox_inches='tight')
plt.show()

## 6) Analisis adicional: Edad y Nacionalidad

Los siguientes graficos utilizan las tablas adicionales descargadas por `fetch_data.py`:
- Tabla 65219: Tasas de paro por sexo y grupo de edad
- Tablas 65086 + 65112: Activos y ocupados por nacionalidad, sexo y grupo de edad (merge para calcular tasa de paro)

In [ ]:
# =====================================================
# CHART 7: Tasa de paro por grupo de edad y sexo
# =====================================================

json_path_edad = DATA_RAW / 'epa_tasas_paro_edad_raw.json'
with open(json_path_edad, encoding='utf-8') as f:
    edad_raw = json.load(f)

# Parse series — structure: "{loc/measure}. {measure/loc}. {sexo}. {edad}."
rows_edad = []
for serie in edad_raw:
    parts = [p.strip() for p in serie['Nombre'].split('.') if p.strip()]
    # sexo is always at index 2, edad at index 3
    if len(parts) < 4:
        continue
    sexo = parts[2]
    edad = parts[3]
    for dp in serie.get('Data', []):
        rows_edad.append({
            'sexo': sexo, 'edad': edad,
            'anyo': dp['Anyo'], 'periodo_id': dp['FK_Periodo'],
            'valor': dp['Valor']
        })

df_edad = pd.DataFrame(rows_edad)
print(f'Sexos: {df_edad["sexo"].unique()}')
print(f'Edades: {df_edad["edad"].unique()}')

# Total age group is "16 y más años", not "Total"
total_age = [e for e in df_edad['edad'].unique() if '16 y m' in e.lower()]
total_age_label = total_age[0] if total_age else None
print(f'Total age label: {total_age_label}')

# Use latest available quarter
latest = df_edad.sort_values(['anyo', 'periodo_id'], ascending=False).iloc[0]
latest_anyo, latest_per = int(latest['anyo']), int(latest['periodo_id'])
periodo_map = {19: 'T4', 20: 'T1', 21: 'T2', 22: 'T3'}
trim_label = f"{periodo_map.get(latest_per, 'P' + str(latest_per))} {latest_anyo}"

df_e = df_edad[
    (df_edad['anyo'] == latest_anyo) &
    (df_edad['periodo_id'] == latest_per) &
    (df_edad['edad'] != total_age_label)
].copy()

# Extract numeric start of age range for sorting
df_e['age_start'] = df_e['edad'].str.extract(r'(\d+)').astype(float)
df_e = df_e.sort_values('age_start')
age_labels = df_e['edad'].unique()

fig, ax = plt.subplots(figsize=(14, 7))
x = np.arange(len(age_labels))
width = 0.25
for i, (sexo, color) in enumerate([
    ('Ambos sexos', '#555555'), ('Hombres', '#3498db'), ('Mujeres', '#e74c3c')
]):
    mask = df_e['sexo'] == sexo
    if mask.sum() == 0:
        mask = df_e['sexo'].str.lower() == sexo.lower()
    vals = df_e.loc[mask].set_index('edad').reindex(age_labels)['valor'].values
    ax.bar(x + (i - 1) * width, vals, width, label=sexo, color=color, alpha=0.85)

ax.set_xticks(x)
short_labels = [e.replace(' años', '').replace('De ', '').replace(' a ', '-')
                .replace(' y más', '+') for e in age_labels]
ax.set_xticklabels(short_labels, rotation=45, ha='right')
ax.set_ylabel('Tasa de paro (%)')
ax.set_xlabel('Grupo de edad')
ax.set_title(f'Tasa de paro por grupo de edad y sexo — Total Nacional ({trim_label})')
ax.legend()
plt.tight_layout()
fig.savefig(CHARTS / '07_paro_por_edad.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =====================================================
# CHART 8: Evolucion del paro juvenil vs total
# =====================================================

# Ensure df_edad and total_age_label exist (load if necessary)
if 'df_edad' not in globals():
    json_path_edad = DATA_RAW / 'epa_tasas_paro_edad_raw.json'
    with open(json_path_edad, encoding='utf-8') as f:
        edad_raw = json.load(f)
    rows_edad = []
    for serie in edad_raw:
        parts = [p.strip() for p in serie['Nombre'].split('.') if p.strip()]
        if len(parts) < 4:
            continue
        sexo = parts[2]
        edad = parts[3]
        for dp in serie.get('Data', []):
            rows_edad.append({
                'sexo': sexo, 'edad': edad,
                'anyo': dp['Anyo'], 'periodo_id': dp['FK_Periodo'],
                'valor': dp['Valor']
            })
    df_edad = pd.DataFrame(rows_edad)

if 'total_age_label' not in globals():
    total_age = [e for e in df_edad['edad'].unique() if '16 y m' in e.lower()]
    total_age_label = total_age[0] if total_age else None

ambos_mask = df_edad['sexo'].str.lower() == 'ambos sexos'

# Identify the two youngest age groups dynamically (exclude the total "16 y más")
non_total_ages = [e for e in df_edad['edad'].unique() if e != total_age_label]
age_groups_sorted = sorted(non_total_ages,
                           key=lambda e: int(''.join(c for c in e if c.isdigit())[:2]) if any(c.isdigit() for c in e) else 999)
youth_groups = age_groups_sorted[:2]

print(f'Youth groups: {youth_groups}')
print(f'Total age: {total_age_label}')

df_juv = df_edad[ambos_mask & df_edad['edad'].isin(youth_groups + [total_age_label])].copy()

# Build date from anyo + periodo_id
q_to_month = {19: 12, 20: 3, 21: 6, 22: 9}
date_strs = [
    f"{int(row['anyo'])}-{q_to_month.get(int(row['periodo_id']), 1):02d}-01"
    for _, row in df_juv.iterrows()
]
df_juv['fecha'] = pd.to_datetime(date_strs)
df_juv = df_juv.sort_values('fecha')

fig, ax = plt.subplots(figsize=(14, 6))
colors = ['#e74c3c', '#f39c12', '#95a5a6']
# use explicit (linestyle, marker) tuples to avoid passing a positional fmt arg
styles = [('-', 'o'), ('-', 'o'), ('--', 's')]
for (edad, color, (linestyle, marker)) in zip(youth_groups + [total_age_label], colors, styles):
    mask = df_juv['edad'] == edad
    label = edad.replace(' años', '').replace('De ', '').replace(' a ', '-').replace(' y más', '+')
    if edad == total_age_label:
        label = f'Total ({label})'
    # Use numpy.asarray to avoid static type-checker issues with Series.to_numpy on mixed types
    x = np.asarray(df_juv.loc[mask, 'fecha'])
    y = np.asarray(df_juv.loc[mask, 'valor'])
    ax.plot(x, y, linestyle=linestyle, marker=marker, label=label, color=color, linewidth=2, markersize=6)

ax.set_xlabel('Fecha')
ax.set_ylabel('Tasa de paro (%)')
ax.set_title(f'Evolucion del paro juvenil vs total — Total Nacional ({PERIOD_LABEL})')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
fig.savefig(CHARTS / '08_paro_juvenil_evolucion.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# =====================================================
# CHART 9: Tasa de paro por edad y nacionalidad
# =====================================================
# Computed from two tables: 65086 (activos) and 65112 (ocupados)
# tasa_paro = (activos - ocupados) / activos * 100

def load_nationality_table(json_path, measure_name):
    with open(json_path, encoding='utf-8') as f:
        raw = json.load(f)
    rows = []
    for serie in raw:
        parts = [p.strip() for p in serie['Nombre'].split('.') if p.strip()]
        if len(parts) < 6:
            continue
        sexo, edad, nacionalidad = parts[2], parts[3], parts[4]
        for dp in serie.get('Data', []):
            rows.append({
                'sexo': sexo, 'edad': edad, 'nacionalidad': nacionalidad,
                'anyo': dp['Anyo'], 'periodo_id': dp['FK_Periodo'],
                'valor': dp['Valor']
            })
    return pd.DataFrame(rows)

activos_path = DATA_RAW / 'epa_activos_nacionalidad_edad_raw.json'
ocupados_path = DATA_RAW / 'epa_ocupados_nacionalidad_edad_raw.json'

df_act = load_nationality_table(activos_path, 'activos')
df_ocu = load_nationality_table(ocupados_path, 'ocupados')

# Merge and compute unemployment rate
merge_keys = ['sexo', 'edad', 'nacionalidad', 'anyo', 'periodo_id']
df_merge = (
    df_act[merge_keys + ['valor']].rename(columns={'valor': 'activos'})
    .merge(df_ocu[merge_keys + ['valor']].rename(columns={'valor': 'ocupados'}),
           on=merge_keys, how='inner')
)
df_merge['tasa_paro'] = (df_merge['activos'] - df_merge['ocupados']) / df_merge['activos'] * 100

# Latest quarter, both sexes, exclude total age, Spanish vs Foreign
latest = df_merge.sort_values(['anyo', 'periodo_id'], ascending=False).iloc[0]
lat_anyo, lat_per = int(latest['anyo']), int(latest['periodo_id'])
trim9 = f"{periodo_map.get(lat_per, 'P' + str(lat_per))} {lat_anyo}"

# Find the 'total' age group (contains '16 y m' or similar)
total_edad = [e for e in df_merge['edad'].unique() if '16 y m' in e.lower()]
total_edad_val = total_edad[0] if total_edad else None

df_c9 = df_merge[
    (df_merge['anyo'] == lat_anyo) &
    (df_merge['periodo_id'] == lat_per) &
    (df_merge['sexo'].str.lower() == 'ambos sexos') &
    (df_merge['edad'] != total_edad_val) &
    (df_merge['nacionalidad'].isin(['Española', 'Extranjera: Total']))
].copy()

# Sort age groups
df_c9['age_start'] = df_c9['edad'].str.extract(r'(\d+)').astype(float)
df_c9 = df_c9.sort_values('age_start')
age_order = df_c9['edad'].unique()

fig, ax = plt.subplots(figsize=(12, 7))
x = np.arange(len(age_order))
width = 0.35

esp = df_c9[df_c9['nacionalidad'] == 'Española'].set_index('edad').reindex(age_order)
ext = df_c9[df_c9['nacionalidad'] == 'Extranjera: Total'].set_index('edad').reindex(age_order)

bars1 = ax.bar(x - width/2, esp['tasa_paro'].values, width, label='Española', color='#2196F3', alpha=0.85)
bars2 = ax.bar(x + width/2, ext['tasa_paro'].values, width, label='Extranjera', color='#FF9800', alpha=0.85)

for bar in list(bars1) + list(bars2):
    h = bar.get_height()
    if not np.isnan(h):
        ax.text(bar.get_x() + bar.get_width()/2, h + 0.3, f'{h:.1f}%', ha='center', va='bottom', fontsize=9)

short_age = [e.replace(' años', '').replace('De ', '').replace(' a ', '-')
             .replace(' y más', '+') for e in age_order]
ax.set_xticks(x)
ax.set_xticklabels(short_age, fontsize=12)
ax.set_title(f'Tasa de paro por grupo de edad y nacionalidad — {trim9}')
ax.set_xlabel('Grupo de edad')
ax.set_ylabel('Tasa de paro (%)')
ax.legend(fontsize=11, loc='upper right')
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f'{v:.0f}%'))
plt.tight_layout()
fig.savefig(CHARTS / '09_paro_edad_nacionalidad.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\nDatos del grafico ({trim9}):')
print(df_c9[['edad', 'nacionalidad', 'tasa_paro']].to_string(index=False))

## 7) Conclusiones

1. **Fuerte desigualdad territorial:** Las provincias del sur (Andalucía, Extremadura, Canarias) mantienen tasas de paro significativamente más altas que las del norte (País Vasco, Navarra, Aragón). Esta brecha se ha mantenido estable a lo largo del periodo analizado. (ver gráfico 1 y 6)

2. **Brecha de género persistente pero reduciéndose:** La tasa de paro femenina es consistentemente más alta que la masculina, aunque la diferencia se ha ido estrechando. (ver gráfico 2)

3. **Economía terciarizada:** El sector servicios domina abrumadoramente el empleo (>75%), con agricultura, industria y construcción en niveles relativamente estables. (ver gráfico 3)

4. **Evolución del empleo total:** El gráfico 5 muestra la trayectoria del empleo total a lo largo del periodo, incluyendo eventos significativos que hayan afectado al mercado laboral. (ver gráfico 5)

5. **Concentración del empleo:** La distribución de empleo por provincia está altamente sesgada — Madrid y Barcelona concentran una proporción desproporcionada, mientras la mayoría de provincias tienen poblaciones ocupadas relativamente pequeñas. (ver gráfico 4)